In [1]:
# Install required libraries
! pip install langchain pdf2image pypdf chromadb llama-cpp-python ctransformers sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.0 MB/s eta 0:00:00
     ━━━━━━

In [2]:
# Let's download related gguf model
! huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmp04bwp7rz
mistral-7b-instruct-v0.2.Q4_K_M.gguf: 100% 4.37G/4.37G [00:24<00:00, 177MB/s]
./mistral-7b-instruct-v0.2.Q4_K_M.gguf


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter ,RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from ctransformers import AutoModelForCausalLM

In [4]:
mistral = "/content/mistral-7b-instruct-v0.2.Q4_K_M.gguf"

In [5]:
def build_llm(name):
  llm = AutoModelForCausalLM.from_pretrained(name ,temperature=0.2,repetition_penalty=1.1,gpu_layers=50,model_type="llama2",max_new_tokens = 1000,context_length = 6000)
  return llm

In [6]:
llm_mistral = build_llm(mistral)

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
path_name = 'pdf1.pdf'
loader = PyPDFLoader(path_name)
documents = loader.load()

In [10]:
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [11]:
!rm -rf chroma_db

In [12]:
# save to disk
db2 = Chroma.from_documents(docs, embeddings, persist_directory="./new_chroma_db")

In [13]:
# loading chroma data from disk
db = Chroma(persist_directory="./new_chroma_db", embedding_function=embeddings)

In [14]:
# Querying data from ChromaDB
query = "Who is the author of the book and what is his connection to the character Mark?"
docs = db2.similarity_search(query,k=7)
docs

[Document(page_content="Donna sees Mark on the boat to Spain. She likes him \nand he likes her. Then Mark sees Donna dancing with \nhis brother, Dave ... Is Donna in love with Mark? \nWhy is Dave dancing with her? \nPenguin Readers are simplified texts designed in association with \nLongman, the world famous educational publisher, to provide a step-by-\nstep approach to the joys of reading for pleasure. The series includes \noriginal stories, contemporary titles based on today's best-selling media \nhits, and easily accessible versions of the literary classics published by \nPenguin around the world. Each book has an introduction and extensive \nactivity material. They are published at seven levels from Easystarts (200 \nwords) to Advanced (3000 words). \nSeries Editors: Andy Hopkins and Jocelyn Potter \n6 Advanced (3000 words) \n5 Upper Intermediate (2300 words) \n4 Intermediate (1700 words) \n3 Pre-lntermediate (1200 words) \n2 Elementary (600 words) \nI Beginner (300 words) \nEasyst

In [15]:
# Making a string of related verses.
verses = ""

for doc in docs:
    verses += doc.page_content + "\n"

In [16]:
verses

"Donna sees Mark on the boat to Spain. She likes him \nand he likes her. Then Mark sees Donna dancing with \nhis brother, Dave ... Is Donna in love with Mark? \nWhy is Dave dancing with her? \nPenguin Readers are simplified texts designed in association with \nLongman, the world famous educational publisher, to provide a step-by-\nstep approach to the joys of reading for pleasure. The series includes \noriginal stories, contemporary titles based on today's best-selling media \nhits, and easily accessible versions of the literary classics published by \nPenguin around the world. Each book has an introduction and extensive \nactivity material. They are published at seven levels from Easystarts (200 \nwords) to Advanced (3000 words). \nSeries Editors: Andy Hopkins and Jocelyn Potter \n6 Advanced (3000 words) \n5 Upper Intermediate (2300 words) \n4 Intermediate (1700 words) \n3 Pre-lntermediate (1200 words) \n2 Elementary (600 words) \nI Beginner (300 words) \nEasystarts (200 words) Contem

In [17]:
template = f"""
Given the following extracted parts of a long document ("SOURCES") and a question ("QUESTION").
Don't try to make up an answer and use the text in the SOURCES only for the answer.
If you don't know the answer, just say that you don't know.
List down all the SOURCES in the response.

QUESTION: {query}
=========
SOURCES:
{verses}
=========
ANSWER:
"""

In [18]:
# Function to trim tokens in a string.
def trim_string_by_tokens(input_string, num_tokens):
    # Split the string into tokens
    tokens = input_string.split()

    # Ensure the number of tokens to keep is not greater than the total number of tokens
    num_tokens = min(num_tokens, len(tokens))

    # Join the desired number of tokens back together
    trimmed_string = ' '.join(tokens[:num_tokens])

    return trimmed_string.replace("/", " ")

In [19]:
final_prompt = (template, 1500)[0]
final_prompt

'\nGiven the following extracted parts of a long document ("SOURCES") and a question ("QUESTION").\nDon\'t try to make up an answer and use the text in the SOURCES only for the answer.\nIf you don\'t know the answer, just say that you don\'t know.\nList down all the SOURCES in the response.\n\nQUESTION: Who is the author of the book and what is his connection to the character Mark?\n=========\nSOURCES:\nDonna sees Mark on the boat to Spain. She likes him \nand he likes her. Then Mark sees Donna dancing with \nhis brother, Dave ... Is Donna in love with Mark? \nWhy is Dave dancing with her? \nPenguin Readers are simplified texts designed in association with \nLongman, the world famous educational publisher, to provide a step-by-\nstep approach to the joys of reading for pleasure. The series includes \noriginal stories, contemporary titles based on today\'s best-selling media \nhits, and easily accessible versions of the literary classics published by \nPenguin around the world. Each boo

In [20]:
prompt1_response_mistral = llm_mistral(final_prompt)

In [21]:
prompt1_response_mistral

'Derek Strange is the author of the book "Girl Meets Boy". In the story, there is no direct connection mentioned between Derek and the character Mark.\n\nSOURCES:\n1. Girl Meets Boy by Derek Strange\n2. Penguin Readers introduction\n3. Exercises section in the book\n4. Comprehension questions section in the book\n5. Author\'s biography at the end of the book.'

In [22]:
print(prompt1_response_mistral.split("\n\nSOURCES:")[0])

Derek Strange is the author of the book "Girl Meets Boy". In the story, there is no direct connection mentioned between Derek and the character Mark.


In [23]:
print(prompt1_response_mistral.split("\n\nSOURCES:")[1])


1. Girl Meets Boy by Derek Strange
2. Penguin Readers introduction
3. Exercises section in the book
4. Comprehension questions section in the book
5. Author's biography at the end of the book.
